In [7]:
from paho.mqtt import client as mqtt_client
import time
from dotenv import dotenv_values

In [8]:
storage=[]
imei_dict={}
device_count=-1
def convert_str_to_dict(data):
    x=data.split(",")
    l=[]
    field,value=x[0][1:5:],int(x[0][6:-1:])
    field=field[1:-1]
    l.append(value)
    for i in range(1,len(x)-1):
        field,value=x[i].split(":")
        field=field[1:-1]
        if(value[0]!='"'):
            if(i==6):
                value=int(value)
            else:
                value=float(value)
        else:
            value=value[1:-1]
        l.append(value)
    field,value=x[-1][0:4:],int(x[-1][5:6:])
    field=field[1:-1]
    value=int(value)
    l.append(value)
    return l
def soc_calculator(EBV_value):
    if(EBV_value>52.95):
        return  90
    elif(EBV_value>52.4):
        return 80
    elif(EBV_value>51.85):
        return 70
    elif(EBV_value>51.3):
        return 60
    elif(EBV_value>50.75):
        return 50
    elif(EBV_value>50.2):
        return 40
    elif(EBV_value>49.65):
        return 30
    elif(EBV_value>49):
        return 20
    elif(EBV_value>48.55):
        return 10
    elif(EBV_value>48):
        return 0
    else:
        return -1
    


In [9]:
# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    #print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("emb/f2s/track/863005061009598",0)
flag=0
info=[]
# The callback for when a PUBLISH message is received from the server.
def to_storage(x,device_count,client):
    if(x[6]==0):
        soc=soc_calculator(x[5])
        x[10]=soc
        if(x[1] not in imei_dict.keys()):
            imei_dict[x[1]] = len(storage)
            storage.append([])
            storage[imei_dict[x[1]]].append(x)
        else:
            ping_count=len(storage[imei_dict[x[1]]])
            storage[imei_dict[x[1]]].append(x)
        print(x[0],x[5],soc,x[6])
        if(len(info)==0):
            info.append(x)
        #on_publish(client,x[1],x[10])
    else:
        if(len(info)==1):
            print("Split")
            info.pop()

def on_message(client, userdata, message):
    #print("#")
    #print(message.topic+" "+str(message.payload))
    x=convert_str_to_dict(str(message.payload.decode('utf-8')))
    #print("#",x)
    flag=to_storage(x,device_count,client)
    #print(storage)
def on_publish(client,imei,soc):
        topic="emb/soc/"+imei
        print(topic)
        info=client.publish(topic=topic,payload=soc,qos=0)
secrets= dotenv_values(".env")


broker_address = secrets["broker_address"]  #Broker address
port = int(secrets["port"])                         #Broker port
user = secrets["user_name"]                   #Connection username
password = secrets["Password"]            #Connection passwordbroker_address= "m11.cloudmqtt.com"  #Broker address

client = mqtt_client.Client("trial")
client.username_pw_set(user, password=password)
client.on_connect = on_connect
client.on_message = on_message



client.connect(broker_address, port)
#client.subscribe("emb/f2s/track/#",0)
# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.


client.loop_forever()

In [ ]:

print(imei_dict)
print(storage)


{'863005061009598': 0}
[[[168872807, '863005061009598', 9.0, 0.0, 0.0, 53.32, 0, 0.86, 11.8, 1.0, 90, 0.0, 0.0, 4], [168872816, '863005061009598', 7.0, 0.0, 0.0, 53.26, 0, 0.86, 11.8, 1.0, 90, 0.0, 0.0, 4], [168872895, '863005061009598', 23.0, 77.634199, 12.914505, 52.07, 0, 5.92, 11.8, '000000000000', 70, 0.0, 4], [168872907, '863005061009598', 19.0, 77.634199, 12.914505, 53.35, 0, 5.92, 11.8, '000000000000', 90, 0.0, 4], [168872919, '863005061009598', 21.0, 77.634199, 12.914505, 53.35, 0, 5.92, 11.8, '000000000000', 90, 0.0, 4], [168872931, '863005061009598', 22.0, 77.634199, 12.914505, 53.37, 0, 5.92, 11.8, '000000000000', 90, 0.0, 4], [168873001, '863005061009598', 21.0, 77.626848, 12.914298, 51.33, 0, 10.98, 11.8, '000000000000', 60, 0.0, 4], [168873013, '863005061009598', 22.0, 77.626848, 12.914298, 53.35, 0, 10.98, 11.8, '000000000000', 90, 0.0, 4], [168873025, '863005061009598', 25.0, 77.626848, 12.914298, 53.35, 0, 10.98, 11.8, '000000000000', 90, 0.0, 4], [168873037, '8630050